In [2]:
# package for handling files and directories
import os
# package for handling the logs history
import wandb
# package for handling logs tabular data
import pandas as pd
# package for handling the environment variables
from dotenv import load_dotenv
# get the environment variables
load_dotenv()

True

In [3]:
path_full = os.environ.get("ORI_PATH")
path_model_log = os.path.join(path_full, "data", "log_train")
wandb_repo = "bugi-sulistiyo-universitas-mulawarman/CAD - Glaucoma Segmentation/"
run_id = os.environ.get("RUN_ID")

In [4]:
runs_id = {}
for element in run_id.split(","):
    key, value = element.split(":")
    runs_id[key] = value
del run_id

In [5]:
os.makedirs(path_model_log, exist_ok=True)

In [6]:
api = wandb.Api()

In [22]:
merge_df = pd.DataFrame()
for model, log_id in runs_id.items():
    run = api.run(f"{wandb_repo}{log_id}")
    log_df = run.history()
    log_df.sort_values(by="_step", inplace=True)
    log_df.drop(columns=["_timestamp"], inplace=True)
    log_df.rename(columns={"_step": "epoch"}, inplace=True)
    log_df = log_df[["epoch",
                    "train_loss", "train_auc", "train_f1", "train_accuracy", "train_precision", "train_recall",
                    "val_loss", "val_auc", "val_f1", "val_accuracy", "val_precision", "val_recall"]]
    log_df.to_csv(os.path.join(path_model_log, f"{model}.csv"), index=False)
    log_df["model"] = model
    merge_df = pd.concat([merge_df, log_df])

merge_df.reset_index(drop=True, inplace=True)
merge_df.to_csv(os.path.join(path_model_log, "merged_log.csv"), index=False)